In [1]:
import os
import sys
import cv2

In [4]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src/data')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src/utils')))

In [6]:
import cv_utils
import file_utils as futils
import data_loader as dload
import yolo_converting as yolo_conv

from config import DATASET_RAW_FOLDER, ANNOTATIONS_FL_DRONES_DS_FOLDER, \
                    YOLO_ANNOTATIONS_PATH, YOLO_IMAGES_PATH

## Store dataset in the required format for YOLO training

In [7]:
files = futils.read_files_as_map('../dataset/raw/Video_1')

In [8]:
files

{0: '../dataset/raw/Video_1/frame_000000.jpg',
 1: '../dataset/raw/Video_1/frame_000001.jpg',
 2: '../dataset/raw/Video_1/frame_000002.jpg',
 3: '../dataset/raw/Video_1/frame_000003.jpg',
 4: '../dataset/raw/Video_1/frame_000004.jpg',
 5: '../dataset/raw/Video_1/frame_000005.jpg',
 6: '../dataset/raw/Video_1/frame_000006.jpg',
 7: '../dataset/raw/Video_1/frame_000007.jpg',
 8: '../dataset/raw/Video_1/frame_000008.jpg',
 9: '../dataset/raw/Video_1/frame_000009.jpg',
 10: '../dataset/raw/Video_1/frame_000010.jpg',
 11: '../dataset/raw/Video_1/frame_000011.jpg',
 12: '../dataset/raw/Video_1/frame_000012.jpg',
 13: '../dataset/raw/Video_1/frame_000013.jpg',
 14: '../dataset/raw/Video_1/frame_000014.jpg',
 15: '../dataset/raw/Video_1/frame_000015.jpg',
 16: '../dataset/raw/Video_1/frame_000016.jpg',
 17: '../dataset/raw/Video_1/frame_000017.jpg',
 18: '../dataset/raw/Video_1/frame_000018.jpg',
 19: '../dataset/raw/Video_1/frame_000019.jpg',
 20: '../dataset/raw/Video_1/frame_000020.jpg',
 2

In [1]:
!rm -rf ../dataset/yolo/labels
!rm -rf ../dataset/yolo/images

In [9]:
dload.create_yolo_dataset_folders()

In [11]:
yolo_conv.process_dataset_to_yolo(DATASET_RAW_FOLDER, ANNOTATIONS_FL_DRONES_DS_FOLDER, os.path.dirname(YOLO_ANNOTATIONS_PATH))

In [22]:
image_path = '../dataset/yolo/images/img_0.jpg'
image = cv2.imread(image_path)
annotations = []
with open('../dataset/yolo/labels/img_0.txt', 'r') as file:
    for line in file:
        # Split the line by spaces and convert to float
        annotation = list(map(float, line.strip().split()))
        annotations.append(annotation)

cv_utils.draw_yolo_annotations(image, annotations)

0.0 0.5427083333333333 0.2552083333333333 0.04791666666666667
-61 249 61 272


In [13]:
!ls -l ../dataset/yolo/images | grep -v '^d' | wc -l
!ls -l ../dataset/yolo/labels | grep -v '^d' | wc -l

20365
20365


In [33]:
image_path = '../dataset/yolo/images/Video_48_5799.jpg'
annotation_path = '../dataset/yolo/labels/Video_48_5799.txt'
target_width = 480
image = cv2.imread(image_path)
h, w = image.shape[:2]

# Read annotations for the current image
with open(annotation_path, 'r') as f:
    annotations = f.readlines()

# Randomly crop the image horizontally
#crop_x = random.randint(0, w - target_width)
crop_x = 0

cropped_image = image[:, crop_x:crop_x + target_width]
_, new_w = cropped_image.shape[:2]

# Adjust annotations based on crop
new_annotations = []
for ann in annotations:
    class_id, x_center, y_center, bbox_width, bbox_height = map(float, ann.split())

    # Convert bbox coordinates
    new_x_center = (x_center * w - crop_x) / new_w
    new_bbox_width = bbox_width * w / new_w
    if cv_utils.obj_out_of_image(new_x_center, new_bbox_width) is None:
        new_annotations.append([class_id, new_x_center, y_center, new_bbox_width, bbox_height])
    else:
        norm_x_center, norm_bbox_width = cv_utils.normalize_coordinates(new_x_center, new_bbox_width)
        if norm_bbox_width > 0 and cv_utils.obj_out_of_image(norm_x_center, norm_bbox_width) is None:
            new_annotations.append([class_id, norm_x_center, y_center, norm_bbox_width, bbox_height])
    
cv_utils.draw_yolo_annotations(cropped_image, new_annotations)

0.596875 0.39375 0.07708333333333334 0.058333333333333334
268 175 305 203


### Crop images to 480x480 since all images have the same heigth 480, randomly crop the width and make another crop if the detection is out of the cropped image

In [35]:
cv_utils.crop_images_and_update_annotations(YOLO_IMAGES_PATH, YOLO_ANNOTATIONS_PATH, 480)

Number of images in dataset: 20364
Number of detections in dataset: 18704


In [37]:
has_detections, no_detections = futils.categorize_files_by_detections(YOLO_ANNOTATIONS_PATH)

In [38]:
print(f"Percentage of files with detections: {(len(has_detections) / (len(has_detections) + len(no_detections)) * 100):.2f}%")

Percentage of files with detections: 88.94%


## Now split dataset into train/test/val folders

In [40]:
dload.split_dataset(YOLO_IMAGES_PATH, YOLO_ANNOTATIONS_PATH, 0.7, .15, .15)

Number of training images: 14254
Number of testing images: 3053
Number of validation images: 3057


In [46]:
dload.create_yaml_file(os.path.dirname(YOLO_IMAGES_PATH))